In [ ]:
import time
from board import SCL, SDA
from busio import I2C
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
import Jetson.GPIO as GPIO
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets
from jetbot import Camera, bgr8_to_jpeg

i2c_bus = I2C(SCL, SDA)
pca = PCA9685(i2c_bus)
pca.frequency = 50

In [ ]:
camera_tilt = servo.Servo(pca.channels[6], min_pulse=500, max_pulse=2600, actuation_range=180)
camera_pan = servo.Servo(pca.channels[7], min_pulse=500, max_pulse=2600, actuation_range=180)
camera_z = servo.Servo(pca.channels[8], min_pulse=500, max_pulse=2600, actuation_range=180)

frnt_rw = servo.ContinuousServo(pca.channels[0])
frnt_rs = servo.Servo(pca.channels[1], min_pulse=500, max_pulse=2600, actuation_range=180)

mid_rw = servo.ContinuousServo(pca.channels[2])
mid_rs = servo.Servo(pca.channels[3], min_pulse=500, max_pulse=2600, actuation_range=180)

back_rw = servo.ContinuousServo(pca.channels[4])
back_rs = servo.Servo(pca.channels[5], min_pulse=500, max_pulse=2600, actuation_range=180)

back_lw = servo.ContinuousServo(pca.channels[9])
back_ls = servo.Servo(pca.channels[10], min_pulse=500, max_pulse=2600, actuation_range=180)

mid_lw = servo.ContinuousServo(pca.channels[11])
mid_ls = servo.Servo(pca.channels[12], min_pulse=500, max_pulse=2600, actuation_range=180)

frnt_lw = servo.ContinuousServo(pca.channels[13])
frnt_ls = servo.Servo(pca.channels[14], min_pulse=500, max_pulse=2600, actuation_range=180)

r_wheels = [frnt_rw, mid_rw, back_rw]
l_wheels = [frnt_lw, mid_lw, back_lw]
a_wheels = [frnt_lw, mid_lw, back_lw, frnt_rw, mid_rw, back_rw]

    

a_steer = [frnt_rs, mid_rs, back_rs, frnt_ls, mid_ls, back_ls]

def para():
    frnt_rs.angle = 85
    mid_rs.angle = 100
    back_rs.angle = 110

    frnt_ls.angle = 75
    mid_ls.angle = 90
    back_ls.angle = 85
    time.sleep(.5)
    

In [ ]:
camera_tilt.angle = 90
camera_pan.angle=90

def go(change):
    
    para()
    time.sleep(.2)
    for wheel in r_wheels:
        wheel.throttle = -1
    for wheel in l_wheels:
        wheel.throttle = 1
    time.sleep(3)
    for wheel in a_wheels:
        wheel.throttle=0
        
def backwardo(change):
    
    para()
    time.sleep(.2)
    for wheel in r_wheels:
        wheel.throttle = 1
    for wheel in l_wheels:
        wheel.throttle = -1
    time.sleep(3)
    for wheel in a_wheels:
        wheel.throttle=0   

def tilt_plus(change):
    if camera_tilt.angle <= 170:
        camera_tilt.angle +=20
def tilt_minus(change):
    if camera_tilt.angle >= 10:
        camera_tilt.angle -= 10
def pan_plus(change):
    if camera_pan.angle <= 170:
        camera_pan.angle += 10
def pan_minus(change):
    if camera_pan.angle >= 10:
        camera_pan.angle -= 10
def stop(change):
    camera_pan.angle = 90
    camera_tilt.angle = 90
    para()
    
def perp_plus(change):
    frnt_rs.angle=0
    mid_rs.angle=10
    back_rs.angle=20

    frnt_ls.angle=160
    mid_ls.angle=180
    back_ls.angle=175
    time.sleep(.5)
    
    for wheel in r_wheels:
        wheel.throttle = 1
    for wheel in l_wheels:
        wheel.throttle = 1
    time.sleep(3)
    for wheel in a_wheels:
        wheel.throttle=0
        
def perp_min(change):
    frnt_rs.angle=0
    mid_rs.angle=10
    back_rs.angle=20

    frnt_ls.angle=160
    mid_ls.angle=180
    back_ls.angle=175
    time.sleep(.5)
    
    for wheel in r_wheels:
        wheel.throttle = -1
    for wheel in l_wheels:
        wheel.throttle = -1
    time.sleep(3)
    for wheel in a_wheels:
        wheel.throttle=0

In [ ]:
# create buttons
button_layout = widgets.Layout(width='80px', height='60px', align_self='center')
stop_button = widgets.Button(description='90', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='tilt+', layout=button_layout)
backward_button = widgets.Button(description='tilt-', layout=button_layout)
left_button = widgets.Button(description='pan+', layout=button_layout)
right_button = widgets.Button(description='pan-', layout=button_layout)
another_right = widgets.Button(description='right', layout=button_layout)
another_left = widgets.Button(description='left', layout=button_layout)
another_top = widgets.Button(description='forward', layout=button_layout)
another_bottom = widgets.Button(description='backward', layout=button_layout)
lateral_right = widgets.Button(description='lat+', layout=button_layout)
lateral_left = widgets.Button(description='lat-', layout=button_layout)
z_up = widgets.Button(description='z-up', layout=button_layout)
z_dwn = widgets.Button(description='z-dwn', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
top_box = widgets.HBox([another_left, forward_button, another_right], layout=widgets.Layout(align_self='center'))
bottom_box = widgets.HBox([lateral_left, backward_button, lateral_right], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([another_top, top_box, middle_box, bottom_box, another_bottom])
display(controls_box)

In [ ]:
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(tilt_minus)
backward_button.on_click(tilt_plus)
left_button.on_click(pan_plus)
right_button.on_click(pan_minus)
another_top.on_click(go)
another_bottom.on_click(backwardo)
lateral_right.on_click(perp_plus)
lateral_left.on_click(perp_min)

In [ ]:
camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=400, height=400)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)